In [1]:
using Revise

using Pkg
Pkg.activate("..")
using FSI

using LinearAlgebra

┌ Info: Recompiling stale cache file /home/fs/.julia/compiled/v1.0/FSI/t27zx.ji for FSI [39fdbc7a-862a-11e9-015b-61723af7393f]
└ @ Base loading.jl:1190


In [2]:
# construct a random block system
A = rand(2,2)
A⁻¹ = inv(A)
B₁ᵀ = rand(2,2)
B₂ = rand(2,2)
M = rand(2,2)
M⁻¹ = inv(M)
G₁ᵀ = rand(2,2)
G₂ = rand(2,2)
T₁ᵀ = rand(2,2)
T₂ = rand(2,2)
O32 = zeros(2,2)
O23 = zeros(2,2)
O22 = zeros(2,2)
# S = [A O32 B₁ᵀ O32; O23 M T₁ᵀ G₁ᵀ; B₂ -T₂ O22 O22; O23 G₂ O22 O22]
S = [A B₁ᵀ O32 O32; B₂ O22 -T₂ O22; O23 T₁ᵀ M G₁ᵀ; O23 O22 G₂ O22]

rċ = rand(2)
rf = rand(2)
ru̇ = rand(2)
rλ = rand(2)
b = [rċ;rf;ru̇;rλ];
b1 = [rċ;rf]
b2 = [ru̇;rλ];

In [3]:
# construct saddle point system
ċ = zeros(2)
f = zeros(2)
u̇ = zeros(2)
λ = zeros(2)
St = SaddleSystem1d((ċ, f, u̇, λ), (A⁻¹, B₁ᵀ, B₂),
                  (M, G₁ᵀ, G₂), (x->T₁ᵀ*x, x->T₂*x))

Saddle system with 2 constraints on fluid and 2 constraints on body
   Fluid state of type Array{Float64,1}
   Fluid force of type Array{Float64,1}
   Body state of type Array{Float64,1}
   Joint force of type Array{Float64,1}


In [4]:
bt = (rċ, rf, ru̇, rλ);
aa,bb,cc,dd = St\bt
x = [aa;bb;cc;dd]

8-element Array{Float64,1}:
  -0.24225919817874653
   0.06659768237625463
  -2.3570372557375325 
   2.8050319134947452 
   1.622365693468431  
  -1.0662224678098442 
 -13.757887704711502  
   7.511000633280043  

In [5]:
S\b

8-element Array{Float64,1}:
  -0.2422591981787409 
   0.06659768237624987
  -2.3570372557375334 
   2.805031913494745  
   1.622365693468431  
  -1.0662224678098442 
 -13.757887704711504  
   7.511000633280049  

### check step 1

In [6]:
x1 = [A B₁ᵀ;B₂ O22]\b1
# norm([A B₁ᵀ;B₂ O22]*x1 - [rċ;rf])

4-element Array{Float64,1}:
   0.573285065650858 
   0.7952643222161414
 -13.245053911422001 
  10.40930297665247  

In [7]:
using ImportMacros
@import ViscousFlow.SaddlePointSystems as Sad

Sf = Sad.SaddleSystem((ċ,f),(A⁻¹,B₁ᵀ,B₂),tol=1e-3,
            issymmetric=false,isposdef=true,store=true,precompile=true)
qq,ww = Sf\(b1[1:2], b1[3:4])

([0.573285, 0.795264], [-13.2451, 10.4093])

### check step 2

In [8]:
inv(B₂*A⁻¹*B₁ᵀ)

2×2 Array{Float64,2}:
 12.8929    0.410464
 -8.86261  -0.454672

In [9]:
ee = [M-T₁ᵀ*inv(B₂*A⁻¹*B₁ᵀ)*T₂ G₁ᵀ;G₂ O22]\[b2[1:2]-T₁ᵀ*ww;b2[3:4]]

4-element Array{Float64,1}:
   1.6223656934684283
  -1.0662224678098442
 -13.757887704711464 
   7.511000633280021 

### check step 3

In [10]:
ċ = qq + A⁻¹*B₁ᵀ*inv(B₂*A⁻¹*B₁ᵀ)*T₂*ee[1:2]

2-element Array{Float64,1}:
 -0.2422591981787403 
  0.06659768237625352

In [11]:
f = f - inv(B₂*A⁻¹*B₁ᵀ)*T₂*ee[1:2]

2-element Array{Float64,1}:
 10.888016655684464
 -7.60427106315772 